In [12]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file from the same directory as this notebook
# In Jupyter, the current working directory is typically where the notebook is located
try:
    env_file = Path('.env')
    if env_file.exists():
        load_dotenv(dotenv_path=env_file, override=True)
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in .env file")
        print(f"✅ Gemini API key loaded from .env file (first 10 chars: {google_api_key[:10]}...)")
        print(f"✅ Verified: GOOGLE_API_KEY is set in environment (length: {len(google_api_key)} chars)")
        print("ℹ️  ADK will use GOOGLE_API_KEY")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

✅ Gemini API key loaded from .env file (first 10 chars: AIzaSyCxBj...)
✅ Verified: GOOGLE_API_KEY is set in environment (length: 39 chars)
ℹ️  ADK will use GOOGLE_API_KEY


In [13]:
import requests
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("Google ADK tools imported successfully")

Google ADK tools imported successfully


In [14]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


In [15]:
def get_fee_for_payment_method(card: str) -> dict:
    database = {
        "card 1": 0.01, # 1%
        "card 2": 0.02, # 2%
        "card 3": 0.03, # 3%
    }

    fee = database.get(card.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"Payment method '{card}' not found",
        }

print("✅ Fee lookup function created")
print(f"💳 Test: {get_fee_for_payment_method('card 1')}")
print(f"💳 Test: {get_fee_for_payment_method('card 5')}")
        

✅ Fee lookup function created
💳 Test: {'status': 'success', 'fee_percentage': 0.01}
💳 Test: {'status': 'error', 'error_message': "Payment method 'card 5' not found"}


In [16]:
def get_exachange_rates(base_curr: str, target_curr: str) -> int:
    url = f"https://api.exchangerate-api.com/v4/latest/{base_curr}"
    response = requests.get(url)
    data = response.json()
    rate = data["rates"].get(target_curr)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_curr}/{target_curr}",
        }

print("✅ Exchange rate function created")
print(f"💱 Test: {get_exachange_rates('USD', 'INR')}")

✅ Exchange rate function created
💱 Test: {'status': 'success', 'rate': 88.75}


In [17]:
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exachange_rates()` to get currency conversion rates and wait here for 15 to 20 seconds to get the real time rate
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_fee_for_payment_method, get_exachange_rates],
)

print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")

✅ Currency agent created with custom function tools
🔧 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


In [18]:
currency_runner = InMemoryRunner(agent=currency_agent)
_ = await currency_runner.run_debug(
    "I want to convert 500 US Dollars to Indian Rupees using my Card 2. How much will I receive?"
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Indian Rupees using my Card 2. How much will I receive?


currency_agent > So, you will receive 4348.75 Indian Rupees for your 500 US Dollars.

Here's the breakdown:

1. **Fee Deduction:** A fee of 0.02% is deducted from your payment. This amounts to 0.1 USD from your 500 USD.
2. **Remaining Amount:** After the fee deduction, you have 499.9 USD remaining for conversion.
3. **Currency Conversion:** With the exchange rate of 1 USD = 88.75 INR, your 499.9 USD is converted to 4348.75 INR.


In [19]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),
)
print(calculation_agent)

name='CalculationAgent' description='' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model=Gemini(model='gemini-2.5-flash-lite', speech_config=None, retry_options=HttpRetryOptions(
  attempts=5,
  exp_base=7.0,
  http_status_codes=[
    429,
    500,
    503,
    504,
  ],
  initial_delay=1.0
)) instruction='You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.\n\n     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.\n\n     **RULES:**\n    1.  Your output MUST be ONLY a Python code block.\n    2.  Do NOT write any text before or after the code block.\n    3.  The Python code MUST calculate the result.\n    4.  The Python code MUST print the final result to stdout.\n    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the co

In [20]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exachange_rates() tool to get the currency conversion rate and wait here for 15 to 20 seconds to get the real time rate
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exachange_rates,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

In [21]:
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to EUR using a Card 3. Show me the precise calculation."
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



 ### Created new session: debug_session_id

User > Convert 1,250 USD to EUR using a Card 3. Show me the precise calculation.


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


enhanced_currency_agent > Here is the precise calculation for converting 1,250 USD to EUR using Card 3:

**Fee Breakdown:**
*   The transaction fee percentage for Card 3 is 3%.
*   The transaction fee amount is 37.50 USD (1,250 USD \* 0.03).
*   The amount remaining after deducting the fee is 1,212.50 USD (1,250 USD - 37.50 USD).

**Conversion:**
*   The exchange rate applied is 1 USD = 0.86 EUR.
*   The final converted amount is 1,042.75 EUR (1,212.50 USD \* 0.86).

Therefore, 1,250 USD is converted to 1,042.75 EUR.
